<a href="https://colab.research.google.com/github/ShaswataJash/kfpcomponent/blob/main/TabularDataPreparationUsingPycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is the development workflow for kubeflow pipeline component of the same name as this notebook. Refer https://github.com/ShaswataJash/kfpcomponent

#Install required softwares

In [1]:
!uname -a

Linux 108a0453d1b0 5.4.188+ #1 SMP Sun Apr 24 10:03:06 PDT 2022 x86_64 x86_64 x86_64 GNU/Linux


In [2]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [3]:
!python --version

Python 3.7.13


In [1]:
!pip install pycaret==2.3.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 22.6 MB/s 
     |████████████████████████████████| 1.7 MB 34.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 61.5 MB/s 
     |████████████████████████████████| 262 kB 59.1 MB/s 
     |████████████████████████████████| 2.0 MB 48.6 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 10.4 MB 59.3 MB/s 
     |████████████████████████████████| 88 kB 8.1 MB/s 
     |████████████████████████████████| 17.8 MB 49.1 MB/s 
     |████████████████████████████████| 120 kB 81.1 MB/s 
     |████████████████████████████████| 6.8 MB 59.8 MB/s 
     |████████████████████████████████| 167 kB 62.3 MB/s 
     |████████████████████████████████

In [2]:
#Ref: https://github.com/pycaret/pycaret/issues/2490
!pip install Jinja2==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 13.9 MB/s 
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!apt-get install ca-certificates fuse tzdata curl unzip && \
  echo "user_allow_other" >> /etc/fuse.conf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fuse is already the newest version (2.9.7-1ubuntu1).
ca-certificates is already the newest version (20211016~18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.18).
tzdata is already the newest version (2022a-0ubuntu0.18.04).
tzdata set to manually installed.
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
!curl https://rclone.org/install.sh | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4497  100  4497    0     0  11442      0 --:--:-- --:--:-- --:--:-- 11442
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/rclone  [binary]
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/README.html  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/git-log.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/rclone.1  [text]  
Purging old database entries in /usr/share/man...
Processing manual pages under /usr/share/man...
Purging old database entries in /usr/share/man/id...
Processing manual pages under /usr/share/man/id...
Purging old database entries

In [5]:
!rclone --version

rclone v1.58.1
- os/version: ubuntu 18.04 (64 bit)
- os/kernel: 5.4.188+ (x86_64)
- os/type: linux
- os/arch: amd64
- go/version: go1.17.9
- go/linking: static
- go/tags: none


#Develop source code files

In [6]:
%%writefile data_preparation.py
#!/usr/bin/env python3
import argparse

# Defining and parsing the command-line arguments
parser = argparse.ArgumentParser(description='kubeflow pipeline component to read csv file and prepare the data')
parser.add_argument('--bypass-rclone-for-input-data', default=False, action="store_true", help='whether input csv file should be read like local file - rclone is completely bypassed')
parser.add_argument('--bypass-rclone-for-output-data', default=False, action="store_true", help='whether output csv file should be written like local file - rclone is completely bypassed')
parser.add_argument('--rclone-environment-var', type=str, default= '{}', help='json formatted key-value pairs of strings which will be set as environment variables before executing rclone commands')
parser.add_argument('--input-datasource-directory-mountable', default=False, action="store_true", help='whether input csv file is present in mountable remote location when rclone is used')
parser.add_argument('--input-datasource-file-name', type=str, default='', help='name of the csv file including file extension and the directory/bucket path holding the specific file(if any) when rclone is used')
parser.add_argument('--additional-options-csv-parsing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.read_csv()')
parser.add_argument('--type-of-data-analysis-task', choices=['classification', 'regression', 'clustering', 'anomaly_detection'])
parser.add_argument('--target-variable-name', type=str, help='for classification and regression, specify the column name holding target variable')
parser.add_argument('--target-emptyindicator', type=str, default='', help='if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc')
parser.add_argument('--data-preparations-options', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pycaret setup() function')
parser.add_argument('--additional-options-csv-writing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.to_csv()')
parser.add_argument('--output-datasource-directory-mountable', default=False, action="store_true", help='whether output csv file will be written in mountable remote location when rclone is used')
parser.add_argument('--output-datasource-file-name', type=str, default='', help='filename of the prepared data including the directory/bucket path holding the specific file(if any) when rclone is used')
parser.add_argument('--input-datasource-local-file-path-when-rclone-bypassed', type=str, default='', help='absolute local path of the input csv file when rclone is NOT used i.e. when bypass-rclone-for-input-data is enabled')
parser.add_argument('--output-datasource-local-file-path-when-rclone-bypassed', type=str, default= '', help='absolute local path of the output csv file when rclone is NOT used i.e. when bypass-rclone-for-output-data is enabled')
args = parser.parse_args()

#sanity check of arguments
if args.bypass_rclone_for_input_data:
    args.input_datasource_directory_mountable = False
    args.input_datasource_file_name = None
else:
    args.input_datasource_local_file_path_when_rclone_bypassed = None

if args.bypass_rclone_for_output_data:
    args.output_datasource_directory_mountable = False
    args.output_datasource_file_name = None
else:
    args.output_datasource_local_file_path_when_rclone_bypassed = None

if args.bypass_rclone_for_input_data and args.bypass_rclone_for_output_data:
    args.rclone_environment_var = '{}'

#setting rclone related env
import os
import json
rclone_config = json.loads(args.rclone_environment_var)
print('rclone_config = (', type(rclone_config), ')', rclone_config)
for item in rclone_config.items():
    os.environ[item[0]] = item[1]

#temporary directory creation
import tempfile
if not args.bypass_rclone_for_input_data:
    local_datastore_read_dir = tempfile.mkdtemp(prefix="my_local_read-")
    print('local_datastore_read_dir:',local_datastore_read_dir)

if not args.bypass_rclone_for_output_data:
    local_datastore_write_dir = tempfile.mkdtemp(prefix="my_local_write-")
    print('local_datastore_write_dir:',local_datastore_write_dir)

#input file handling
import subprocess
import sys
import ntpath
if args.input_datasource_directory_mountable:
    input_data_read_cmd = "rclone -v mount remoteread:" + ntpath.dirname(args.input_datasource_file_name) + ' ' + local_datastore_read_dir + ' --daemon'
else:
    input_data_read_cmd = "rclone -v copy remoteread:" + args.input_datasource_file_name + ' ' + local_datastore_read_dir
print(input_data_read_cmd)
input_data_read_call = subprocess.run(input_data_read_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(input_data_read_call.stdout)
if input_data_read_call.returncode != 0:
    print("Error in rclone, errorcode=", input_data_read_call.returncode)
    sys.stdout.flush()
    sys.exit("Forceful exit as rclone returned error in context of reading")

#output file handling
if args.output_datasource_directory_mountable:
    output_data_write_cmd = "rclone -v mount remotewrite:" + ntpath.dirname(args.output_datasource_file_name) + ' ' + local_datastore_write_dir + ' --daemon'
    print(output_data_write_cmd)
    output_data_write_call = subprocess.run(output_data_write_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(output_data_write_call.stdout)
    if output_data_write_call.returncode != 0:
        print("Error in rclone, errorcode=", output_data_write_call.returncode)
        sys.stdout.flush()
        sys.exit("Forceful exit as rclone returned error in context of mounted writing")

#handling input csv file reading
import pandas
try:
    parse_config = json.loads(args.additional_options_csv_parsing)
    parse_config['filepath_or_buffer'] =  args.input_datasource_local_file_path_when_rclone_bypassed \
        if args.bypass_rclone_for_input_data else os.path.join(local_datastore_read_dir,ntpath.basename(args.input_datasource_file_name))
    print('parse_config = (', type(parse_config), ')', parse_config)
    my_data = pandas.read_csv(**parse_config)
    print(my_data)
    
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while parsing input csv file")

#handling data preprocessing
import pycaret
try:
    if os.path.exists("logs.log"):
        os.remove("logs.log") #removing any content from log which pycaret will internally use for its own logging
    print('pycaret version = ', pycaret.utils.version())
    setup_config = json.loads(args.data_preparations_options)
    if args.type_of_data_analysis_task == 'classification':
        import pycaret.classification
        setup_fn = pycaret.classification.setup
        get_config_fn = pycaret.classification.get_config
        setup_config['target'] = args.target_variable_name
        
    elif args.type_of_data_analysis_task == 'regression':
        import pycaret.regression
        setup_fn = pycaret.regression.setup
        get_config_fn = pycaret.regression.get_config
        setup_config['target'] = args.target_variable_name

    elif args.type_of_data_analysis_task == 'clustering':
        import pycaret.clustering
        setup_fn = pycaret.clustering.setup
        get_config_fn = pycaret.clustering.get_config

    elif args.type_of_data_analysis_task == 'anomaly':
        import pycaret.anomaly
        setup_fn = pycaret.anomaly.setup
        get_config_fn = pycaret.anomaly.get_config
        
    #as part of pycaret's data cleaning the rows with target column = nan are not being cleaned up. Thus, cleaning those rows explicitely
    if len(args.target_emptyindicator) > 0:
        #ref: https://stackoverflow.com/questions/49291740/delete-rows-if-there-are-null-values-in-a-specific-column-in-pandas-dataframe
        import numpy as np
        my_data[args.target_variable_name] = my_data[args.target_variable_name].replace(args.target_emptyindicator, np.nan)
        my_data = my_data.dropna(axis=0, subset=[args.target_variable_name])

    setup_config['data'] = my_data
    setup_config['log_experiment'] = False
    setup_config['data_split_shuffle'] = False
    setup_config['html'] = False
    setup_config['silent'] = True
    print('setup_config = (', type(setup_config), ')', setup_config)
    setup_fn(**setup_config)
    #ref: https://www.kdnuggets.com/2020/11/5-things-doing-wrong-pycaret.html
    X_transformed = get_config_fn('X')
    my_transformed_data = X_transformed
    if args.type_of_data_analysis_task == 'classification' or args.type_of_data_analysis_task == 'regression':
        y_transformed = get_config_fn('y')
        my_transformed_data = X_transformed.merge(y_transformed,left_index=True, right_index=True)
    
    print("====== PREPARED DATA ====")
    print(my_transformed_data)
    print("=========================")

    pycaret.utils.get_system_logs() #this will print the pycaret's own log into console
    
except BaseException as err:
    pycaret.utils.get_system_logs()
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while transforming input dataframe")

#handling output csv file writing
try:
    to_csv_config = json.loads(args.additional_options_csv_writing)
    to_csv_config['path_or_buf'] = args.output_datasource_local_file_path_when_rclone_bypassed \
        if args.bypass_rclone_for_output_data else os.path.join(local_datastore_write_dir,ntpath.basename(args.output_datasource_file_name))
    print('to_csv_config = (', type(to_csv_config), ')', to_csv_config)
    my_transformed_data.to_csv(**to_csv_config)
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while trying to write prepared data")

if args.bypass_rclone_for_output_data:
    sys.stdout.flush()
    sys.exit(0)

if not args.output_datasource_directory_mountable:
    output_data_write_cmd = "rclone -v copy " + os.path.join(local_datastore_write_dir,ntpath.basename(args.output_datasource_file_name)) \
        + " remotewrite:" + args.output_datasource_file_name
    print(output_data_write_cmd)
    output_data_write_call = subprocess.run(output_data_write_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(output_data_write_call.stdout)
    if output_data_write_call.returncode != 0:
        print("Error in rclone, errorcode=", output_data_write_call.returncode)
        sys.stdout.flush()
        sys.exit("Forceful exit as rclone returned error in context of writing final csv file (copy mode)")

Writing data_preparation.py


Docker size reduction tips:


*   https://devopscube.com/reduce-docker-image-size/
*   https://www.ecloudcontrol.com/best-practices-to-reduce-docker-images-size/



In [7]:
%%writefile Dockerfile
FROM python:3.7.13-slim

RUN python3 -m pip install pycaret==2.3.10
#installing jinja2 additionally due to Ref: https://github.com/pycaret/pycaret/issues/2490
RUN python3 -m pip install Jinja2==3.1.2

#install fuse as dependency for rclone. Additionally, install curl, unzip for rclone installer to work
#libgomp1 installation for pycaret in python-slim
RUN apt-get update \
    && apt-get install --no-install-recommends -y curl fuse libgomp1 unzip \
    && echo "user_allow_other" >> /etc/fuse.conf \
    && curl https://rclone.org/install.sh | bash \
    && apt-get -y remove --purge curl unzip \
    && rm -rf /var/lib/apt/lists/* \
    && rclone --version

COPY src/data_preparation.py /tmp
COPY tests/test_validation.py /tmp
COPY run_tests.sh /tmp
RUN chmod 544 /tmp/run_tests.sh

Writing Dockerfile


In [8]:
%%writefile run_tests.sh
#!/bin/bash

mkdir /tmp/my_local_dir_for_test

#Test: csv reading source from http, rclone read in copy
python /tmp/data_preparation.py --rclone-environment-var '{"RCLONE_CONFIG_REMOTEREAD_TYPE":"http", "RCLONE_CONFIG_REMOTEREAD_URL":"https://raw.githubusercontent.com/pycaret/datasets/main/data/common/"}' \
    --input-datasource-file-name 'CTG.csv' --additional-options-csv-parsing '{"sep":"," , "header":0}' \
    --type-of-data-analysis-task 'classification' --target-variable-name 'NSP' \
    --data-preparations-options '{"ignore_low_variance":true, "remove_outliers":true, "remove_multicollinearity":true, "multicollinearity_threshold":0.7}' \
    --bypass-rclone-for-output-data --output-datasource-local-file-path-when-rclone-bypassed '/tmp/my_local_dir_for_test/CTG_data-prep.csv' \
    --additional-options-csv-writing '{"index":false}'

#https://registry.opendata.aws/humor-detection/
#Test: csv reading source from s3(AWS provider), rclone read in mount
python /tmp/data_preparation.py --rclone-environment-var '{"RCLONE_CONFIG_REMOTEREAD_TYPE":"s3", "RCLONE_CONFIG_REMOTEREAD_PROVIDER":"AWS", "RCLONE_CONFIG_REMOTEREAD_REGION":"us-west-2"}' \
    --input-datasource-directory-mountable --input-datasource-file-name 'humor-detection-pds/Non-humours-biased.csv' \
    --type-of-data-analysis-task 'classification' --target-variable-name 'label' \
    --data-preparations-options '{"preprocess":false, "ignore_features":["image_url"]}' \
    --bypass-rclone-for-output-data --output-datasource-local-file-path-when-rclone-bypassed '/tmp/my_local_dir_for_test/Non-humours-biased_data-prep.csv' \
    --additional-options-csv-writing '{"index":false}'

python /tmp/test_validation.py

Writing run_tests.sh


In [9]:
%%writefile test_validation.py
#!/usr/bin/env python3
import pandas
df = pandas.read_csv(filepath_or_buffer = '/tmp/my_local_dir_for_test/CTG_data-prep.csv')
assert len(df.index) == 2126 #original data had 2129 rows, amongst that 3 rows have no target
assert df.isnull().sum().sum() == 0 #pycaret will remove all missing values

df = pandas.read_csv(filepath_or_buffer = '/tmp/my_local_dir_for_test/Non-humours-biased_data-prep.csv')
assert len(df.columns) == 3 #original data had 4 columns, amongst that one will be dropped
print ('test-validation done successfully')

Writing test_validation.py


Refer: https://github.com/RealOrangeOne/docker-rclone-mount/blob/master/docker-compose.yml

If we have to use mount feature of rclone, it needs to have fuse support in underneath linux kernel. For that we are adding SYS_ADMIN in capability. But note without using mount feature also, we can do testing. in that case, rclone will use only copy feature.

In [10]:
%%writefile docker-compose.test.yml
services:
  sut:
    build: .
    command: /tmp/run_tests.sh
    cap_add:
      - SYS_ADMIN
    security_opt:
      - apparmor:unconfined
    devices:
      - "/dev/fuse:/dev/fuse"

Writing docker-compose.test.yml



*   https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#designing-a-pipeline-component
*   https://github.com/kubeflow/pipelines/blob/sdk/release-1.8/sdk/python/kfp/dsl/types.py
*   https://kubeflow-pipelines.readthedocs.io/en/stable/_modules/kfp/components/_structures.html



In [68]:
%%writefile component_both_input_output_as_artifact.yaml
name: TabularDataPreparationUsingPycaretWhereBothInputOutputAsArtifact
description: |
    Prepare tabular data (csv file) using pycaret library. (For pycaret's data pre-processing capabilities, refer https://pycaret.gitbook.io/docs/get-started/preprocessing)
    Refer data-preparations-options in command line arguments. 
    pycaret internally uses pandas dataframe to read and write csv file. You can utilize options exposed by panda's read_csv() and to_csv(). 
    Refer additional-options-csv-parsing and additional-options-csv-writing in command line arguments
    Input and output csv files are stored in input and output artifacts. Thus the csv files are read or written like locally mounted POSIX files.

inputs:
- name: additional_options_csv_parsing
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pandas.read_csv()'
  optional: true
- name: type_of_data_analysis_task
  type: String
  description: 'choice amongst classification, regression, clustering, anomaly_detection'
- name: target_variable_name 
  type: String
  description: 'for classification and regression, specify the column name holding target variable'
  optional: true
- name: target_emptyindicator
  type: String
  description: 'if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc'
  optional: true
- name: data_preparations_options
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pycaret setup() function'
  optional: true
- name: additional_options_csv_writing
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pandas.to_csv()'
  optional: true
- name: input_datasource_local_file_path_when_rclone_bypassed
  description: 'absolute local path of the input csv file when rclone is NOT used i.e. when input csv file is stored in input artifact of pipeline engine (e.g. argo)'

outputs:
- name: output_datasource_local_file_path_when_rclone_bypassed
  description: 'absolute local path of the output csv file when rclone is NOT used i.e. when output csv file is stored in output artifact of pipeline engine (e.g. argo)'

implementation:
  container:
    image: shasjash/kfpcomponents:TabularDataPreparationUsingPycaret_devlatest
    command:
    - python3 
    - /tmp/data_preparation.py
    args:
    - --bypass-rclone-for-input-data
    - --bypass-rclone-for-output-data
    - if:
        cond: {isPresent: additional_options_csv_parsing}
        then:
        - --additional-options-csv-parsing
        - {inputValue: additional_options_csv_parsing}
    - --type-of-data-analysis-task 
    - {inputValue: type_of_data_analysis_task}
    - if:
        cond: {isPresent: target_variable_name}
        then:
        - --target-variable-name
        - {inputValue: target_variable_name}
    - if:
        cond: {isPresent: target_emptyindicator}
        then:
        - --target-emptyindicator
        - {inputValue: target_emptyindicator}
    - if:
        cond: {isPresent: data_preparations_options}
        then:
        - --data-preparations-options
        - {inputValue: data_preparations_options}
    - if:
        cond: {isPresent: additional_options_csv_writing}
        then:
        - --additional-options-csv-writing
        - {inputValue: additional_options_csv_writing}
    - --input-datasource-local-file-path-when-rclone-bypassed
    - {inputPath: input_datasource_local_file_path_when_rclone_bypassed}
    - --output-datasource-local-file-path-when-rclone-bypassed
    - {outputPath: output_datasource_local_file_path_when_rclone_bypassed}

Overwriting component_both_input_output_as_artifact.yaml


In [69]:
%%writefile component_input_using_rclone_output_as_artifact.yaml
name: TabularDataPreparationUsingPycaretWhereInputUsingRcloneOutputAsArtifact
description: |
    Prepare tabular data (csv file) using pycaret library. (For pycaret's data pre-processing capabilities, refer https://pycaret.gitbook.io/docs/get-started/preprocessing)
    Refer data-preparations-options in command line arguments. 
    pycaret internally uses pandas dataframe to read and write csv file. You can utilize options exposed by panda's read_csv() and to_csv(). 
    Refer additional-options-csv-parsing and additional-options-csv-writing in command line arguments
    Input csv files can be stored in rclone compatible storage. Both mount and copy mode are supported. (refer: https://rclone.org/)
    rclone configurations have to be shared through environment variables (refer: https://rclone.org/docs/#environment-variables). 
    Create rclone read configuration file name as 'REMOTEREAD' . Because the same is used within code.
    So convention for creating any environment variables related to rclone-read should start with 'RCLONE_CONFIG_REMOTEREAD'.
    Output csv files are stored in output artifacts. Thus the csv files are written like locally mounted POSIX files.

inputs:
- name: rclone_environment_var
  type: String
  description: 'json formatted key-value pairs of strings which will be set as environment variables before executing rclone commands'
- name: input_datasource_directory_mountable
  type: Boolean 
  description: 'whether input csv file is present in mountable remote location  when rclone is used'
  optional: true
- name: input_datasource_file_name
  type: String
  description: 'name of the csv file including file extension and the directory/bucket path holding the specific file(if any)  when rclone is used'
- name: additional_options_csv_parsing
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pandas.read_csv()'
  optional: true
- name: type_of_data_analysis_task
  type: String
  description: 'choice amongst classification, regression, clustering, anomaly_detection'
- name: target_variable_name 
  type: String
  description: 'for classification and regression, specify the column name holding target variable'
  optional: true
- name: target_emptyindicator
  type: String
  description: 'if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc'
  optional: true
- name: data_preparations_options
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pycaret setup() function'
  optional: true
- name: additional_options_csv_writing
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pandas.to_csv()'
  optional: true

outputs:
- name: output_datasource_local_file_path_when_rclone_bypassed
  description: 'absolute local path of the output csv file when rclone is NOT used i.e. when output csv file is stored in output artifact of pipeline engine (e.g. argo)'

implementation:
  container:
    image: shasjash/kfpcomponents:TabularDataPreparationUsingPycaret_devlatest
    command:
    - python3 
    - /tmp/data_preparation.py
    args:
    - --bypass-rclone-for-output-data
    - --rclone-environment-var
    - {inputValue: rclone_environment_var}
    - if:
        cond: {isPresent: input_datasource_directory_mountable}
        then:
        - --input-datasource-directory-mountable
        - {inputValue: input_datasource_directory_mountable}
    - --input-datasource-file-name
    - {inputValue: input_datasource_file_name}
    - if:
        cond: {isPresent: additional_options_csv_parsing}
        then:
        - --additional-options-csv-parsing
        - {inputValue: additional_options_csv_parsing}
    - --type-of-data-analysis-task 
    - {inputValue: type_of_data_analysis_task}
    - if:
        cond: {isPresent: target_variable_name}
        then:
        - --target-variable-name
        - {inputValue: target_variable_name}
    - if:
        cond: {isPresent: target_emptyindicator}
        then:
        - --target-emptyindicator
        - {inputValue: target_emptyindicator}
    - if:
        cond: {isPresent: data_preparations_options}
        then:
        - --data-preparations-options
        - {inputValue: data_preparations_options}
    - if:
        cond: {isPresent: additional_options_csv_writing}
        then:
        - --additional-options-csv-writing
        - {inputValue: additional_options_csv_writing}
    - --output-datasource-local-file-path-when-rclone-bypassed
    - {outputPath: output_datasource_local_file_path_when_rclone_bypassed}

Overwriting component_input_using_rclone_output_as_artifact.yaml


In [70]:
%%writefile component_input_as_artifact_output_using_rclone.yaml
name: TabularDataPreparationUsingPycaretWhereInputAsArtifactOutputUsingRclone
description: |
    Prepare tabular data (csv file) using pycaret library. (For pycaret's data pre-processing capabilities, refer https://pycaret.gitbook.io/docs/get-started/preprocessing)
    Refer data-preparations-options in command line arguments. 
    pycaret internally uses pandas dataframe to read and write csv file. You can utilize options exposed by panda's read_csv() and to_csv(). 
    Refer additional-options-csv-parsing and additional-options-csv-writing in command line arguments
    Output csv files can be stored in rclone compatible storage. Both mount and copy mode are supported. (refer: https://rclone.org/)
    rclone configurations have to be shared through environment variables (refer: https://rclone.org/docs/#environment-variables). 
    Create rclone write configuration file name as 'REMOTEWRITE'. Because the same is used within code.
    So convention for creating any environment variables related to rclone should start either with 'RCLONE_CONFIG_REMOTEWRITE'.
    Intput csv files are stored in intput artifacts. Thus the csv files are read like locally mounted POSIX files.

inputs:
- name: rclone_environment_var
  type: String
  description: 'json formatted key-value pairs of strings which will be set as environment variables before executing rclone commands'
- name: additional_options_csv_parsing
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pandas.read_csv()'
  optional: true
- name: type_of_data_analysis_task
  type: String
  description: 'choice amongst classification, regression, clustering, anomaly_detection'
- name: target_variable_name 
  type: String
  description: 'for classification and regression, specify the column name holding target variable'
  optional: true
- name: target_emptyindicator
  type: String
  description: 'if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc'
  optional: true
- name: data_preparations_options
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pycaret setup() function'
  optional: true
- name: additional_options_csv_writing
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pandas.to_csv()'
  optional: true
- name: output_datasource_directory_mountable
  type: Boolean
  description: 'whether output csv file will be written in mountable remote location  when rclone is used'
  optional: true
- name: output_datasource_file_name
  type: String 
  description: 'filename of the prepared data including the directory/bucket path holding the specific file(if any) when rclone is used'
- name: input_datasource_local_file_path_when_rclone_bypassed
  description: 'absolute local path of the input csv file when rclone is NOT used i.e. when input csv file is stored in input artifact of pipeline engine (e.g. argo)'

implementation:
  container:
    image: shasjash/kfpcomponents:TabularDataPreparationUsingPycaret_devlatest
    command:
    - python3 
    - /tmp/data_preparation.py
    args:
    - --bypass-rclone-for-input-data
    - --rclone-environment-var
    - {inputValue: rclone_environment_var}
    - if:
        cond: {isPresent: additional_options_csv_parsing}
        then:
        - --additional-options-csv-parsing
        - {inputValue: additional_options_csv_parsing}
    - --type-of-data-analysis-task 
    - {inputValue: type_of_data_analysis_task}
    - if:
        cond: {isPresent: target_variable_name}
        then:
        - --target-variable-name
        - {inputValue: target_variable_name}
    - if:
        cond: {isPresent: target_emptyindicator}
        then:
        - --target-emptyindicator
        - {inputValue: target_emptyindicator}
    - if:
        cond: {isPresent: data_preparations_options}
        then:
        - --data-preparations-options
        - {inputValue: data_preparations_options}
    - if:
        cond: {isPresent: additional_options_csv_writing}
        then:
        - --additional-options-csv-writing
        - {inputValue: additional_options_csv_writing}
    - if:
        cond: {isPresent: output_datasource_directory_mountable}
        then:
        - --output-datasource-directory-mountable
        - {inputValue: output_datasource_directory_mountable}
    - if:
        cond: {isPresent: output_datasource_file_name}
        then:
        - --output-datasource-file-name
        - {inputValue: output_datasource_file_name}
    - --input-datasource-local-file-path-when-rclone-bypassed
    - {inputPath: input_datasource_local_file_path_when_rclone_bypassed}

Overwriting component_input_as_artifact_output_using_rclone.yaml


In [71]:
%%writefile component_both_input_output_using_rclone.yaml
name: TabularDataPreparationUsingPycaretWhereBothInputOutputUsingRclone
description: |
    Prepare tabular data (csv file) using pycaret library. (For pycaret's data pre-processing capabilities, refer https://pycaret.gitbook.io/docs/get-started/preprocessing)
    Refer data-preparations-options in command line arguments. 
    pycaret internally uses pandas dataframe to read and write csv file. You can utilize options exposed by panda's read_csv() and to_csv(). 
    Refer additional-options-csv-parsing and additional-options-csv-writing in command line arguments
    Input and output csv files can be stored in rclone compatible storage. Both mount and copy mode are supported. (refer: https://rclone.org/)
    rclone configurations have to be shared through environment variables (refer: https://rclone.org/docs/#environment-variables). 
    Create rclone read and write configuration file name as 'REMOTEREAD' and 'REMOTEWRITE'. Because the same are used within code.
    So convention for creating any environment variables related to rclone should start either with 'RCLONE_CONFIG_REMOTEREAD' or 'RCLONE_CONFIG_REMOTEWRITE'.

inputs:
- name: rclone_environment_var
  type: String
  description: 'json formatted key-value pairs of strings which will be set as environment variables before executing rclone commands'
- name: input_datasource_directory_mountable
  type: Boolean 
  description: 'whether input csv file is present in mountable remote location  when rclone is used'
  optional: true
- name: input_datasource_file_name
  type: String
  description: 'name of the csv file including file extension and the directory/bucket path holding the specific file(if any)  when rclone is used'
- name: additional_options_csv_parsing
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pandas.read_csv()'
  optional: true
- name: type_of_data_analysis_task
  type: String
  description: 'choice amongst classification, regression, clustering, anomaly_detection'
- name: target_variable_name 
  type: String
  description: 'for classification and regression, specify the column name holding target variable'
  optional: true
- name: target_emptyindicator
  type: String
  description: 'if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc'
  optional: true
- name: data_preparations_options
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pycaret setup() function'
  optional: true
- name: additional_options_csv_writing
  type: String
  description: 'json formatted key-value pairs of strings which will be passed to pandas.to_csv()'
  optional: true
- name: output_datasource_directory_mountable
  type: Boolean
  description: 'whether output csv file will be written in mountable remote location  when rclone is used'
  optional: true
- name: output_datasource_file_name
  type: String 
  description: 'filename of the prepared data including the directory/bucket path holding the specific file(if any) when rclone is used'

implementation:
  container:
    image: shasjash/kfpcomponents:TabularDataPreparationUsingPycaret_devlatest
    command:
    - python3 
    - /tmp/data_preparation.py
    args:
    - --rclone-environment-var
    - {inputValue: rclone_environment_var}
    - if:
        cond: {isPresent: input_datasource_directory_mountable}
        then:
        - --input-datasource-directory-mountable
        - {inputValue: input_datasource_directory_mountable}
    - --input-datasource-file-name
    - {inputValue: input_datasource_file_name}
    - if:
        cond: {isPresent: additional_options_csv_parsing}
        then:
        - --additional-options-csv-parsing
        - {inputValue: additional_options_csv_parsing}
    - --type-of-data-analysis-task 
    - {inputValue: type_of_data_analysis_task}
    - if:
        cond: {isPresent: target_variable_name}
        then:
        - --target-variable-name
        - {inputValue: target_variable_name}
    - if:
        cond: {isPresent: target_emptyindicator}
        then:
        - --target-emptyindicator
        - {inputValue: target_emptyindicator}
    - if:
        cond: {isPresent: data_preparations_options}
        then:
        - --data-preparations-options
        - {inputValue: data_preparations_options}
    - if:
        cond: {isPresent: additional_options_csv_writing}
        then:
        - --additional-options-csv-writing
        - {inputValue: additional_options_csv_writing}
    - if:
        cond: {isPresent: output_datasource_directory_mountable}
        then:
        - --output-datasource-directory-mountable
        - {inputValue: output_datasource_directory_mountable}
    - --output-datasource-file-name
    - {inputValue: output_datasource_file_name}

Overwriting component_both_input_output_using_rclone.yaml


#Software testing

Now let us simulate testing what will be done by docker-hub infrastructure as part of auto-testing by using docker-compose.test.yml present in github source repository.

In [15]:
!rm -rf /tmp/my_local_dir_for_test
!chmod 544 run_tests.sh
!cp data_preparation.py /tmp
!cp test_validation.py /tmp
!./run_tests.sh

rclone_config = ( <class 'dict'> ) {'RCLONE_CONFIG_REMOTEREAD_TYPE': 'http', 'RCLONE_CONFIG_REMOTEREAD_URL': 'https://raw.githubusercontent.com/pycaret/datasets/main/data/common/'}
local_datastore_read_dir: /tmp/my_local_read-xzsjxnwb
rclone -v copy remoteread:CTG.csv /tmp/my_local_read-xzsjxnwb
2022/06/22 12:48:42 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
2022/06/22 12:48:43 INFO  : CTG.csv: Copied (new)
2022/06/22 12:48:43 INFO  : 
Transferred:   	  277.715 KiB / 277.715 KiB, 100%, 0 B/s, ETA -
Transferred:            1 / 1, 100%
Elapsed time:         0.2s


parse_config = ( <class 'dict'> ) {'sep': ',', 'header': 0, 'filepath_or_buffer': '/tmp/my_local_read-xzsjxnwb/CTG.csv'}
          FileName  ...  NSP
0     Variab10.txt  ...  2.0
1       Fmcs_1.txt  ...  1.0
2       Fmcs_1.txt  ...  1.0
3       Fmcs_1.txt  ...  1.0
4       Fmcs_1.txt  ...  1.0
...            ...  ...  ...
2124  S8001045.dsp  ...  2.0
2125  S8001045.dsp  ...  1.0
2126       

In [16]:
! pip3 install kfp==1.8.12

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 301 kB 31.6 MB/s 
     |████████████████████████████████| 106 kB 54.4 MB/s 
     |████████████████████████████████| 1.6 MB 57.2 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 87 kB 7.6 MB/s 
     |████████████████████████████████| 76 kB 6.4 MB/s 
  Created wheel for kfp: filename=kfp-1.8.12-py3-none-any.whl size=419048 sha256=c0ec05eb5a064e69f2643f278ba0fb531e96204134bfd191b9b55dc1739a7a52
  Stored in directory: /root/.cache/pip/wheels/54/0c/4a/3fc55077bc88cc17eacaae34c5fd3f6178c1d16d2ee3b0afdf
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=45ba519b13ece37936cbb7a02d52e24be29429214d38983a490d59d58aad195a
  Stored in directory: /root/.cache/pip/whe

First validate the component.yaml file in http://www.yamllint.com/. Once component.yaml file is corrected, execute the below cell to finally check

In [72]:
import kfp

csv_data_prepare_op_out_to_artifact = kfp.components.load_component_from_file('component_input_using_rclone_output_as_artifact.yaml')
csv_data_prepare_op_both_in_out_artifact = kfp.components.load_component_from_file('component_both_input_output_as_artifact.yaml')

@kfp.dsl.pipeline(name="mysamplepipeline")
def my_sample_pipeline():
    csv_prepared_file_step1 = csv_data_prepare_op_out_to_artifact(rclone_environment_var='{"RCLONE_CONFIG_REMOTEREAD_TYPE":"http", "RCLONE_CONFIG_REMOTEREAD_URL":"https://raw.githubusercontent.com/pycaret/datasets/main/data/common/"}',
                                            input_datasource_file_name='CTG.csv',
                                            type_of_data_analysis_task='classification',
                                            target_variable_name='NSP',
                                            data_preparations_options='{"ignore_low_variance":true, "remove_outliers":true}', 
                                            additional_options_csv_writing='{"index":false}'
                                            ).outputs['output_datasource_local_file_path_when_rclone_bypassed']

    csv_data_prepare_op_both_in_out_artifact(input_datasource_local_file_path_when_rclone_bypassed = csv_prepared_file_step1,
                                            type_of_data_analysis_task='classification',
                                            target_variable_name='NSP',
                                            data_preparations_options='{"remove_multicollinearity":true, "multicollinearity_threshold":0.7}', 
                                            additional_options_csv_writing='{"index":false}'
                                            )


kfp.compiler.Compiler().compile(
    pipeline_func=my_sample_pipeline,
    package_path='my_sample_pipeline.yaml')


In [73]:
kfp.v2.compiler.Compiler().compile(
    pipeline_func=my_sample_pipeline,
    package_path='my_sample_pipeline_v2.json')

/usr/local/lib/python3.7/dist-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


#Push the code to github

Before commiting code to github, install github client (gh) by following instruction mentioned in https://github.com/cli/cli/blob/trunk/docs/install_linux.md (Choose Debian, Ubuntu Linux way of installation) 

Use the colab's 'Terminal' icon present in left vertical pane to open linux terminal to type commands. Once 'gh' is installed, type **$gh auth login** (refer https://docs.github.com/en/get-started/getting-started-with-git/caching-your-github-credentials-in-git) to follow onscreen prompts. For colab, use **Paste an authentication token** option. Personal tokens can be generated in https://github.com/settings/tokens

You can use Shift+Ctrl+v shortcut to paste any string in colab console

In [83]:
!pwd

/content


In [84]:
!rm -Rf kfpcomponent

In [85]:
!git clone https://github.com/ShaswataJash/kfpcomponent.git

Cloning into 'kfpcomponent'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 209 (delta 115), reused 131 (delta 59), pack-reused 0
Receiving objects: 100% (209/209), 99.76 KiB | 4.53 MiB/s, done.
Resolving deltas: 100% (115/115), done.


Follow directory structure according to https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#organizing-the-component-files

In [86]:
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret/src
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret/tests

mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret’: File exists
mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret/src’: File exists
mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret/tests’: File exists


In [87]:
#it will ensure file is coped in git repo only if file content is changed by checking checksum of file content
!rsync -c data_preparation.py kfpcomponent/TabularDataPreparationUsingPycaret/src
!rsync -c component_both_input_output_as_artifact.yaml kfpcomponent/TabularDataPreparationUsingPycaret/component_both_input_output_as_artifact.yaml
!rsync -c component_both_input_output_using_rclone.yaml kfpcomponent/TabularDataPreparationUsingPycaret/component_both_input_output_using_rclone.yaml
!rsync -c component_input_as_artifact_output_using_rclone.yaml kfpcomponent/TabularDataPreparationUsingPycaret/component_input_as_artifact_output_using_rclone.yaml
!rsync -c component_input_using_rclone_output_as_artifact.yaml kfpcomponent/TabularDataPreparationUsingPycaret/component_input_using_rclone_output_as_artifact.yaml
!rsync -c test_validation.py kfpcomponent/TabularDataPreparationUsingPycaret/tests
!rsync -c Dockerfile kfpcomponent/TabularDataPreparationUsingPycaret/
!rsync -c run_tests.sh kfpcomponent/TabularDataPreparationUsingPycaret/
!rsync -c docker-compose.test.yml kfpcomponent/TabularDataPreparationUsingPycaret/

In [88]:
%cd kfpcomponent

/content/kfpcomponent


In [89]:
!git add -A

In [90]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   TabularDataPreparationUsingPycaret/component_both_input_output_as_artifact.yaml
	modified:   TabularDataPreparationUsingPycaret/component_both_input_output_using_rclone.yaml
	modified:   TabularDataPreparationUsingPycaret/component_input_as_artifact_output_using_rclone.yaml
	modified:   TabularDataPreparationUsingPycaret/component_input_using_rclone_output_as_artifact.yaml



For git-user who has set their email visibility as private, git provides alternate email address to use in web-based Git operations, e.g., edits and merges. The alias email can be viewed in https://github.com/settings/emails

In [91]:
!git config --global user.email "29448766+ShaswataJash@users.noreply.github.com"

In [92]:
!git commit -a -m "corrected docker url"

[main 7fdb8cd] corrected docker url
 4 files changed, 4 insertions(+), 4 deletions(-)


In [93]:
!git push origin main

Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 1.58 KiB | 1.58 MiB/s, done.
Total 7 (delta 6), reused 0 (delta 0)
remote: Resolving deltas: 100% (6/6), completed with 4 local objects.
To https://github.com/ShaswataJash/kfpcomponent.git
   5877369..7fdb8cd  main -> main


In [94]:
%cd ..

/content
